# Project FOCS - Kiva Dataset

In [3]:
import pandas as pd
import numpy as np
import re
import random
import datetime as dt
import qgrid
import time
import matplotlib.pyplot as plt
import seaborn as sns

# Guidelines and tasks

You have to work on the Kiva dataset. Some information on the datasets are on the Kaggle web page.

All groups and individual must do the following:
- Normalize the loan_lenders table. In the normalized table, each row must have one loan_id and one lender.
- For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time. If any of those two dates is missing, also the duration must be missing.
- Find the lenders that have funded at least twice.
- For each country, compute how many loans have involved that country as borrowers.
- For each country, compute the overall amount of money borrowed.
- Like the previous point, but expressed as a percentage of the overall amount lent.
- Like the three previous points, but split for each year (with respect to disburse time).
- For each lender, compute the overall amount of money lent. For each loan that has more than one lender, you must assume that all lenders contributed the same amount.
- For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed. Since the country of the lender is often unknown, you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known.
- Which country has the highest ratio between the difference computed at the previous point and the population?
- Which country has the highest ratio between the difference computed at point 9 and the population that is not below the poverty line?
- For each year, compute the total amount of loans. Each loan that has planned expiration time and disburse time in different years must have its amount distributed proportionally to the number of days in each year. For example, a loan with disburse time December 1st, 2016, planned expiration time January 30th 2018, and amount 5000USD has an amount of 5000USD * 31 / (31+365+30) = 363.85 for 2016, 5000USD * 365 / (31+365+30) = 4284.04 for 2017, and 5000USD * 30 / (31+365+30) = 352.11 for 2018.

### Task 1 

Normalize the loan_lenders table. In the normalized table, each row must have one loan_id and one lender.

In [5]:
# The direct link to the Kaggle data set
loans_lenders = pd.read_csv('additional-kiva-snapshot/loans_lenders.csv')
loans_lenders.head(5)

,loan_id,lenders
0,483693,"muc888, sam4326, camaran3922, lachheb1865, reb..."
1,483738,"muc888, nora3555, williammanashi, barbara5610,..."
2,485000,"muc888, terrystl, richardandsusan8352, sherri4..."
3,486087,"muc888, james5068, rudi5955, daniel9859, don92..."
4,534428,"muc888, niki3008, teresa9174, mike4896, david7..."


In [6]:
start_time = time.time()

loans_lenders_norm = (loans_lenders.set_index(['loan_id'])
                      .apply(lambda x: x.str.split(',').explode())
                      .reset_index())

end_time = time.time()
print('Tempo di esecuzione:',end_time-start_time)

Tempo di esecuzione: 18.28634023666382


In [7]:
start_time = time.time()

#Using the explode function to normalize the table to find a faster alternative
loans_lenders_norm = loans_lenders.assign(lenders=loans_lenders['lenders'].str.split(',')).explode('lenders')
loans_lenders_norm = loans_lenders_norm.reset_index(drop=True)

end_time = time.time()
print('Tempo di esecuzione:',end_time-start_time)

Tempo di esecuzione: 15.740858793258667


In [8]:
loans_lenders_norm.head(2)

,loan_id,lenders
0,483693,muc888
1,483693,sam4326


### Task 2 
For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time. If any of those two dates is missing, also the duration must be missing.

In [9]:
loans = pd.read_csv('additional-kiva-snapshot/loans.csv', parse_dates = ['planned_expiration_time', 'disburse_time'])

In [10]:
#Verify columns format
print('Formato planned_expiration_time:',loans['planned_expiration_time'].dtype)
print('Formato disburse_time:',loans['disburse_time'].dtype)

Formato planned_expiration_time: datetime64[ns, UTC]
Formato disburse_time: datetime64[ns, UTC]


In [11]:
#Creation of column 'duration'
loans['duration'] = (loans['planned_expiration_time']-loans['disburse_time']).dt.days

In [12]:
check = loans[['duration','planned_expiration_time','disburse_time']]
check[check.planned_expiration_time.isnull() == True]

,duration,planned_expiration_time,disburse_time
64,NaN,NaT,2010-11-18 08:00:00+00:00
65,NaN,NaT,2010-11-25 08:00:00+00:00
66,NaN,NaT,2010-11-25 08:00:00+00:00
67,NaN,NaT,2010-11-16 08:00:00+00:00
69,NaN,NaT,2010-11-10 08:00:00+00:00
...,...,...,...
1418885,NaN,NaT,2010-04-09 07:00:00+00:00
1418886,NaN,NaT,2010-03-26 07:00:00+00:00
1418887,NaN,NaT,2010-04-07 07:00:00+00:00
1418888,NaN,NaT,2010-04-14 07:00:00+00:00


In [13]:
loans['duration'].head()

0    53.0
1    96.0
2    37.0
3    34.0
4    57.0
Name: duration, dtype: float64

### Task 3
Find the lenders that have funded at least twice.

In [14]:
#Find funded loans
loans_funded = loans[loans['status'] == 'funded']
#Merge with lenders table by loan ID
loans_funded_merged = pd.merge(loans_funded['loan_id'], loans_lenders_norm, on='loan_id')
#Normalize lenders name which sometimes have space in the field
loans_funded_merged['lenders'] = loans_funded_merged['lenders'].str.strip()
#Identify lenders with more than one loan funded
loans_funded_twice = loans_funded_merged['lenders'].value_counts()
loans_funded_twice[loans_funded_twice >= 2]

gooddogg1           167512
trolltech4460       141268
gmct                123164
nms                  96713
themissionbeltco     74067
                     ...  
douglas3061              2
kalsoom6555              2
tara37509030             2
jason2566                2
matthew4252              2
Name: lenders, Length: 893981, dtype: int64

In [15]:
#Verify value counts
loans_funded_merged['lenders'].value_counts().loc[lambda x: x >= 2]

gooddogg1           167512
trolltech4460       141268
gmct                123164
nms                  96713
themissionbeltco     74067
                     ...  
douglas3061              2
kalsoom6555              2
tara37509030             2
jason2566                2
matthew4252              2
Name: lenders, Length: 893981, dtype: int64

### Task 4
For each country, compute how many loans have involved that country as borrowers.

In [16]:
#Idetify the column name
loans.columns 

Index(['loan_id', 'loan_name', 'original_language', 'description',
       'description_translated', 'funded_amount', 'loan_amount', 'status',
       'activity_name', 'sector_name', 'loan_use', 'country_code',
       'country_name', 'town_name', 'currency_policy',
       'currency_exchange_coverage_rate', 'currency', 'partner_id',
       'posted_time', 'planned_expiration_time', 'disburse_time',
       'raised_time', 'lender_term', 'num_lenders_total',
       'num_journal_entries', 'num_bulk_entries', 'tags', 'borrower_genders',
       'borrower_pictured', 'repayment_interval', 'distribution_model',
       'duration'],
      dtype='object')

In [17]:
#Computing the values counts
loans['country_name'].str.strip().value_counts()

Philippines         285336
Kenya               143699
Peru                 86000
Cambodia             79701
El Salvador          64037
                     ...  
Papua New Guinea         1
Botswana                 1
Uruguay                  1
Mauritania               1
Canada                   1
Name: country_name, Length: 96, dtype: int64

### Task 5
For each country, compute the overall amount of money borrowed.

In [18]:
#Compute the overall amount of loans value
loans.groupby('country_name')['loan_amount'].sum().sort_values(ascending=False)

country_name
Philippines       97984600.0
Peru              79437775.0
Kenya             66735975.0
Paraguay          53964700.0
Cambodia          51613525.0
                     ...    
Virgin Islands       10000.0
Vanuatu               9250.0
Botswana              8000.0
Uruguay               8000.0
Gaza                  5000.0
Name: loan_amount, Length: 96, dtype: float64

### Task 6
Like the previous point, but expressed as a percentage of the overall amount lent.

In [19]:
(loans.groupby('country_name')['loan_amount'].sum()/ loans['funded_amount'].sum() * 100).sort_values(ascending=False)

country_name
Philippines       8.669783
Peru              7.028740
Kenya             5.904871
Paraguay          4.774855
Cambodia          4.566820
                    ...   
Virgin Islands    0.000885
Vanuatu           0.000818
Botswana          0.000708
Uruguay           0.000708
Gaza              0.000442
Name: loan_amount, Length: 96, dtype: float64

### Task 7
Like the three previous points, but split for each year (with respect to disburse time).

In [20]:
#Add one column with the reference year
loans['disburse_year'] = pd.DatetimeIndex(loans['disburse_time']).year

In [21]:
(loans.groupby(['disburse_year','country_name'])['loan_amount'].sum() / 
loans.groupby(['disburse_year'])['funded_amount'].sum() *100)

disburse_year  country_name
2005.0         Bulgaria         1.798736
               Cambodia         1.579971
               Ecuador          4.861449
               Gaza             4.861449
               Honduras        32.377248
                                 ...    
2018.0         Tanzania         1.853412
               Thailand         3.434645
               Uganda           2.819001
               Uruguay          1.036874
               Zambia           1.811289
Length: 748, dtype: float64

### Task 8
For each lender, compute the overall amount of money lent. For each loan that has more than one lender, you must assume that all lenders contributed the same amount.

In [22]:
#compute the number of lenders in each loans using loans ID
lenders_in_loan = loans_lenders_norm.groupby('loan_id').size().reset_index(name = 'counts')
#Merge to the df the loans amount and the funded amount by loans ID
lenders_in_loan = pd.merge(lenders_in_loan, loans[['loan_id', 'funded_amount']], on = 'loan_id')
#computing the average contribution for each loan considering the effective amount funded
lenders_in_loan['average_funded'] = lenders_in_loan['funded_amount'] / lenders_in_loan['counts']

In [23]:
lenders_in_loan.head()

,loan_id,counts,funded_amount,average_funded
0,84,3,500.0,166.666667
1,85,2,500.0,250.000000
2,86,3,500.0,166.666667
3,88,3,300.0,100.000000
4,89,4,500.0,125.000000


In [24]:
#Normalize lenders name which sometimes have space in the field
loans_lenders_norm['lenders'] = loans_lenders_norm['lenders'].str.strip()
#Merge the lenders_in_loan table with the loans_lenders table in loan ID
lenders_avg_loans = pd.merge(loans_lenders_norm, lenders_in_loan[['loan_id', 'average_funded']], on = 'loan_id')
#Compute the overall amount of money lent by each lenders
lended_by_lenders = lenders_avg_loans.groupby('lenders')['average_funded'].sum().reset_index(name = 'amount_lended')

In [25]:
lended_by_lenders.head(8)

,lenders,amount_lended
0,000,1703.868411
1,00000,1379.750248
2,0002,2472.563566
3,00mike00,52.631579
4,0101craign0101,2623.565117
5,0132575,117.678140
6,0154884,136.342116
7,0161130,74.779412


### Task 9
For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed. 

Since the country of the lender is often unknown, you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known.

In [26]:
lenders = pd.read_csv('additional-kiva-snapshot/lenders.csv')

In [27]:
#Compute the number of missing country_code 
lenders['country_code'].isnull().value_counts()

True     1458635
False     890539
Name: country_code, dtype: int64

In [28]:
#Known country_code distribution
lenders['country_code'].dropna().value_counts()

US    591612
CA     67970
GB     38380
AU     37103
DE     16007
       ...  
NU         1
IO         1
KM         1
MS         1
GW         1
Name: country_code, Length: 234, dtype: int64

In [29]:
def fill_lenders(lenders):
    random.seed(42)
    country_code = pd.unique(lenders['country_code'].dropna())
    #Definying probability according to known country code distribution
    probs = lenders['country_code'].dropna().value_counts() / len(lenders['country_code'].dropna())
    #Number of missing country_code
    n = len(lenders[pd.isnull(lenders['country_code'])])
    #Indexing missing country_code
    index = lenders.index[lenders['country_code'].isnull()]
    random_elements = pd.Series(np.random.choice(country_code, p = probs, size = n), index = index)
    lenders['country_code'].fillna(random_elements, inplace = True)
    return(lenders)

lenders = fill_lenders(lenders)

In [30]:
#Verify that all missing where replaced
lenders['country_code'].isnull().value_counts()

False    2349174
Name: country_code, dtype: int64

In [31]:
#Identify the lender's name column
lenders.head()

,permanent_name,display_name,city,state,country_code,member_since,occupation,loan_because,loan_purchase_num,invited_by,num_invited
0,qian3013,Qian,NaN,NaN,US,1461300457,NaN,NaN,1.0,NaN,0
1,reena6733,Reena,NaN,NaN,US,1461300634,NaN,NaN,9.0,NaN,0
2,mai5982,Mai,NaN,NaN,US,1461300853,NaN,NaN,NaN,NaN,0
3,andrew86079135,Andrew,NaN,NaN,SK,1461301091,NaN,NaN,5.0,Peter Tan,0
4,nguyen6962,Nguyen,NaN,NaN,US,1461301154,NaN,NaN,NaN,NaN,0


In [32]:
#Merge lended_by_lenders table and lenders by lenders name
df_merged = pd.merge(lended_by_lenders, lenders[['permanent_name', 'country_code']], left_on='lenders', right_on='permanent_name')
#Group amount lended by country 
lended_by_country = df_merged.groupby('country_code')['amount_lended'].sum().reset_index(name='lent')
#Compute the amount of money received by each country
received_by_country = loans.groupby(['country_code'])['funded_amount'].sum().reset_index(name='received')
#Merge lended_by_country table and received_by_country table by country code 
received_lended_by_country = pd.merge(lended_by_country, received_by_country, on = 'country_code', how = 'outer')
received_lended_by_country.fillna(0, inplace = True)
#Compute the column 'difference' as difference between money lent and money borrowed
received_lended_by_country['difference'] = received_lended_by_country['lent'] - received_lended_by_country['received']

In [33]:
received_lended_by_country.sort_values(by='difference',ascending=False)

,country_code,lent,received,difference
216,US,6.832401e+08,36365340.0,6.468748e+08
35,CA,7.668173e+07,50000.0,7.663173e+07
13,AU,5.893864e+07,0.0,5.893864e+07
72,GB,4.760091e+07,0.0,4.760091e+07
52,DE,3.210768e+07,0.0,3.210768e+07
...,...,...,...,...
108,KH,1.109534e+05,50829425.0,-5.071847e+07
172,PY,1.131823e+05,53715200.0,-5.360202e+07
106,KE,3.441059e+05,63650255.0,-6.330615e+07
161,PE,1.275406e+05,78702800.0,-7.857526e+07


### Task 10
Which country has the highest ratio between the difference computed at the previous point and the population?

In [34]:
country_stats = pd.read_csv('additional-kiva-snapshot/country_stats.csv')

In [35]:
country_stats.head(2)

,country_name,country_code,country_code3,continent,region,population,population_below_poverty_line,hdi,life_expectancy,expected_years_of_schooling,mean_years_of_schooling,gni,kiva_country_name
0,India,IN,IND,Asia,Southern Asia,1339180127,21.9,0.623559,68.322,11.696590,6.298834,5663.474799,India
1,Nigeria,NG,NGA,Africa,Western Africa,190886311,70.0,0.527105,53.057,9.970482,6.000000,5442.901264,Nigeria


In [36]:
#Merge the received_lended_by_country table anche country_stats table by Country code
country_ratio = pd.merge(received_lended_by_country, country_stats[['population', 'country_code']], on = 'country_code')
#Compute the new column 'ratio'
country_ratio['ratio'] = country_ratio['difference'] / country_ratio['population']

In [37]:
country_ratio.sort_values(by='ratio',ascending=False)

,country_code,lent,received,difference,population,ratio
103,MT,8.543569e+06,0.0,8.543569e+06,430835,19.830259
114,NO,1.838847e+07,0.0,1.838847e+07,5305383,3.466002
72,IS,9.370578e+05,0.0,9.370578e+05,335025,2.796979
7,AU,5.893864e+07,0.0,5.893864e+07,24450561,2.410523
24,CA,7.668173e+07,50000.0,7.663173e+07,36624199,2.092380
...,...,...,...,...,...,...
101,MN,9.789312e+04,14896700.0,-1.479881e+07,3075647,-4.811608
145,SV,5.314386e+04,37881675.0,-3.782853e+07,6377853,-5.931233
3,AM,9.925993e+04,20284675.0,-2.018542e+07,2930450,-6.888162
127,PY,1.131823e+05,53715200.0,-5.360202e+07,6811297,-7.869576


The country with the highest ratio is Norway followed by Iceland

### Task 11
Which country has the highest ratio between the difference computed at point 9 and the population that is not below the poverty line?

In [38]:
ratio_by_country = pd.merge(country_ratio, country_stats[['population_below_poverty_line', 'country_code']], on = 'country_code')
ratio_by_country['ratio_below'] = ratio_by_country['difference'] / (ratio_by_country['population'].sub(ratio_by_country['population_below_poverty_line']))

In [39]:
ratio_by_country.sort_values(by='ratio_below',ascending=False)

,country_code,lent,received,difference,population,ratio,population_below_poverty_line,ratio_below
103,MT,8.543569e+06,0.0,8.543569e+06,430835,19.830259,16.3,19.831009
24,CA,7.668173e+07,50000.0,7.663173e+07,36624199,2.092380,9.4,2.092380
159,US,6.832401e+08,36365340.0,6.468748e+08,324459463,1.993700,15.1,1.993700
136,SE,1.483943e+07,0.0,1.483943e+07,9910701,1.497314,15.0,1.497316
113,NL,2.429432e+07,0.0,2.429432e+07,17035938,1.426063,8.8,1.426064
...,...,...,...,...,...,...,...,...
133,SA,9.109979e+05,0.0,9.109979e+05,32938213,0.027658,NaN,NaN
134,SB,6.316326e+03,806275.0,-7.999587e+05,611343,-1.308527,NaN,NaN
137,SG,2.880627e+06,0.0,2.880627e+06,5708844,0.504590,NaN,NaN
142,SO,6.957046e+03,308725.0,-3.017680e+05,14742523,-0.020469,NaN,NaN


The country with the highest ratio is Canada

### Task 12
For each year, compute the total amount of loans. Each loan that has planned expiration time and disburse time in different years must have its amount distributed proportionally to the number of days in each year.

For example, a loan with disburse time December 1st, 2016, planned expiration time January 30th 2018, and amount 5000USD has an amount of 5000USD * 31 / (31+365+30) = 363.85 for 2016, 5000USD * 365 / (31+365+30) = 4284.04 for 2017, and 5000USD * 30 / (31+365+30) = 352.11 for 2018.

In [160]:
#Creazione del df con le variabili di interesse
day_loans = loans[['loan_id','loan_amount','planned_expiration_time','disburse_time']]
#Creazione della variabile differenza e degli anni exp e disburse
day_loans['diff']=day_loans['planned_expiration_time'].dt.year-day_loans['disburse_time'].dt.year
day_loans.sort_values(by='diff',ascending=False)
#Eliminazione dei record con valori nulli nelle variabili di data
day_loans = day_loans.dropna(subset=['diff'])
day_loans = day_loans.loc[day_loans["planned_expiration_time"] > day_loans["disburse_time"]]

In [161]:
#Split into diffent df if same year between disburse and expiration
same = day_loans[day_loans["disburse_time"].dt.year == day_loans["planned_expiration_time"].dt.year]
different = day_loans[day_loans["disburse_time"].dt.year != day_loans["planned_expiration_time"].dt.year]
same['year'] = same['disburse_time'].dt.year
day_loans_same = pd.DataFrame(same.groupby('year')['loan_amount'].sum())
day_loans_same

,loan_amount
year,
2012,103911725.0
2013,98427750.0
2014,120644250.0
2015,131208475.0
2016,133271575.0
2017,144870625.0
2018,85300.0


In [175]:
def func(x):
    df = x.loc[x.index.repeat(x["planned_expiration_time"].dt.year - x["disburse_time"].dt.year + 1)]
    df["count"] = df.groupby("loan_id").cumcount()
    df["count_year"] = pd.to_datetime(df.groupby("loan_id").cumcount() + df["disburse_time"].dt.year, format = "%Y")
    df["end_year"] = df["disburse_time"].dt.year.apply(lambda x: dt.datetime(x, 12, 31))
    df["planned_expiration_time"] = df["planned_expiration_time"].dt.tz_localize(None)
    df["disburse_time"] = df["disburse_time"].dt.tz_localize(None)
    df = df.reset_index()
    df.loc[df["count_year"].dt.year == df["disburse_time"].dt.year, "useful_days"] = (df["end_year"] - df["disburse_time"]).dt.days
    df.loc[df["count_year"].dt.year != df["disburse_time"].dt.year, "useful_days"] = (df["planned_expiration_time"] - df["count_year"]).dt.days
    df.loc[(df["useful_days"] > 365) & (df["count_year"].dt.is_leap_year == True), "useful_days"] = 366
    df.loc[(df["useful_days"] > 365) & (df["count_year"].dt.is_leap_year == False), "useful_days"] = 365
    days = pd.DataFrame(df.groupby("loan_id")["useful_days"].sum())
    final = pd.merge(df, days, on = "loan_id", suffixes = ("","_tot"))
    final["true_amount"] = final["loan_amount"]*final["useful_days"] / final["useful_days_tot"]
    final = pd.DataFrame(final.groupby("count_year")["true_amount"].sum()).reset_index()
    final["count_year"] = final["count_year"].dt.year
    return final

In [213]:
day_loans_diff = func(different)

In [214]:
final = pd.merge(day_loans_diff,day_loans_same, left_on = "count_year", right_on="year",how='outer')
final= final.fillna(0)
final["amount_per_year"] = final["true_amount"] + final["loan_amount"]
final = final.drop(['true_amount',"loan_amount"],axis=1)
final = final.rename(columns={'count_year':'Year'})
final = final.set_index('Year')
final.round()

,amount_per_year
Year,
2011,606406.0
2012,113069487.0
2013,123496941.0
2014,150643338.0
2015,155188329.0
2016,154029554.0
2017,165535118.0
2018,5008377.0
